In [1]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
from importlib import reload
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
from keras.callbacks import Callback
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Input
from keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay, prune_low_magnitude

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.model import save_model, load_model
from src.harness.pruning import create_pruning_callbacks, create_pruning_parameters
from src.harness.training import train, TrainingRound
from src.harness.utils import count_params, set_seed

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


## Parameters

In [3]:
X_train, X_test, Y_train, Y_test = load_and_process_mnist()

num_epochs: int = 10
batch_size: int = 60
input_shape: tuple = X_train[0].shape
num_classes: int = 10

num_train_samples: int = X_train.shape[0]

end_step: int = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epochs

pruning_parameters: dict = create_pruning_parameters(0.01, 0, end_step, 100)

## Building

In [4]:
# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
def create_lenet_300_100(
    input_shape: tuple[int, ...], 
    num_classes: int, 
    optimizer = C.OPTIMIZER,
    ) -> keras.Model:
    """
    Function for creating LeNet-300-100 model.

    :param input_shape: Expected input shape for images.
    :param num_classes: Number of potential classes to predict.
    :param optimizer:   Optimizer to use for training.

    :returns: Compiled LeNet-300-100 architecture.
    """
    model = Sequential([
        Input(input_shape),
        Dense(300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(num_classes, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform()),
    ], name="LeNet-300-100")

    model.compile(
        loss=keras.losses.CategoricalCrossentropy(), 
        optimizer=optimizer(), 
        metrics=['accuracy'])

    return model

def global_pruning_nn(
    input_shape: tuple[int, ...], 
    num_classes: int, 
    pruning_parameters: dict,
    optimizer = C.OPTIMIZER,
    ) -> keras.Model:
    """
    Function to define the architecture of a neural network model
    following 300 100 architecture for MNIST dataset and using
    provided parameter which are used to prune the model.
    
    :param input_shape:        Expected input shape for images.
    :param num_classes:        Number of potential classes to predict.
    :param pruning_parameters: Dictionary to be unpacked for the pruning schedule.
    :param optimizer:          Optimizer to use for training.

    :returns: Compiled LeNet-300-100 architecture with layerwise low magnitude pruning.
    """
    model = sparsity.prune_low_magnitude(Sequential([
        Input(input_shape),
        Dense(300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(num_classes, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform())
    ], name="Pruned_LeNet-300-100"), **pruning_parameters)

    model.compile(
        loss=keras.losses.CategoricalCrossentropy(), 
        optimizer=optimizer(), 
        metrics=['accuracy'])
    
    return model

set_seed(0)

## Pruning

In [5]:
def initialize_mask_model(model: keras.Model):
    """
    Function which performs layerwise initialization of a keras model's weights to all 1s
    as an initialization step for masking.

    :param model: Keras model being set to all 1s.
    """
    for weights in model.trainable_weights:
        weights.assign(
            tf.ones_like(
                input=weights,
                dtype=tf.float32,
            )
        )

## Training

In [6]:
from src.harness.experiment import ExperimentData
from src.harness.training import test_step, train, train_one_step, training_loop, TrainingRound

# Create model and masked model
mask_model = global_pruning_nn(input_shape, num_classes, pruning_parameters)
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
initialize_mask_model(mask_model)

print(f'Nonzero parameters before training:')
print(count_params(model)[1])

experiment_data: ExperimentData = ExperimentData()
model(X_train)

Nonzero parameters before training:
266200


<tf.Tensor: shape=(60000, 1, 10), dtype=float32, numpy=
array([[[0.03675784, 0.08286017, 0.15588324, ..., 0.10381736,
         0.15198708, 0.05716838]],

       [[0.0516306 , 0.10735245, 0.10834687, ..., 0.10771041,
         0.16217342, 0.07541302]],

       [[0.12167772, 0.10012673, 0.11473323, ..., 0.0871617 ,
         0.11975257, 0.07545463]],

       ...,

       [[0.05520871, 0.07962444, 0.16368584, ..., 0.08866281,
         0.10203359, 0.09554213]],

       [[0.06297164, 0.0989567 , 0.13472585, ..., 0.10705641,
         0.1076025 , 0.06750833]],

       [[0.07248026, 0.08178942, 0.13388856, ..., 0.1040113 ,
         0.0931598 , 0.09374413]]], dtype=float32)>

In [7]:
# Define the optimizer outside of the function
optimizer = C.OPTIMIZER()

In [8]:
# # Test single step of training
# loss, accuracy = train_one_step(model, mask_model, X_train, Y_train, optimizer)
# print(f'Loss: {loss:.6f}, Accuracy: {accuracy:.6g}')

In [9]:
# Try getting test accuracies with `test_step`
test_loss: tf.keras.metrics.Metric = C.LOSS_FUNCTION()
test_accuracy: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

loss, accuracy = test_step(model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 2.3444080352783203
Test Accuracy: 0.09109999984502792


2024-04-21 20:03:36.340773: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [10]:
# Testing `training_loop` function
# epochs: int = 500
# batch_size: int = len(X_train)
# trained_model, training_round = training_loop(0, model, mask_model, load_and_process_mnist, epochs, batch_size, optimizer=optimizer)
# print(f'Took {np.sum(training_round.test_accuracies != 0)} / {epochs} epochs')
# print(f'Ended with a best training accuracy of {np.max(training_round.train_accuracies) * 100:.2f}% and test accuracy of training accuracy of {np.max(training_round.test_accuracies) * 100:.2f}%')

In [11]:
# Testing `train` function

# Create model and masked model
mask_model = global_pruning_nn(input_shape, num_classes, pruning_parameters)
model: keras.Model = create_lenet_300_100(input_shape, num_classes)
initialize_mask_model(mask_model)

trained_model, pruned_mask_model, training_round = train(0, 0, model, mask_model, load_and_process_mnist)

Early stopping initiated
Directory 'models/model_0/trial0/weights' already exists.
Directory 'models/model_0/trial0/masks' already exists.


In [12]:
test_loss: tf.keras.metrics.Metric = C.LOSS_FUNCTION()
test_accuracy: tf.keras.metrics.Metric = tf.keras.metrics.CategoricalAccuracy()

loss, accuracy = test_step(trained_model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 1.0927822589874268
Test Accuracy: 0.6111000180244446


In [14]:
# Test loading the model back
loaded_model: keras.Model = load_model(0, 0)

loss, accuracy = test_step(loaded_model, X_test, Y_test, test_loss, test_accuracy)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 1.0927822589874268
Test Accuracy: 0.6111000180244446


In [13]:
print(f'Nonzero parameters after training:')
print(count_params(model)[1])

Nonzero parameters after training:
266610
